In [7]:
import pandas as pd
import numpy as np

In [8]:
def normalize_1NF(table, column):
    df = table.copy()
    df[column]= df[column].apply(lambda s: str(s).split(',') if ',' in str(s) else str(s).split('/'))
    lens = list(map(len,df[column].values))
    df_1NF = pd.DataFrame({'ID': np.repeat(df['ID'], lens), 
                           column: np.concatenate(df[column].values), 
                        }) 
    df_1NF[column] = df_1NF[column].str.strip()    
    df_1NF = df_1NF[['ID', column]]
   
    return df_1NF


In [9]:
# scope and extract the relevant data
df_pre = pd.read_excel("./data/Pre_DA.xlsx")
pre_Qs = list(df_pre.columns)

df_post = pd.read_excel("./data/Post_DA.xlsx")
post_Qs = list(df_post.columns)

# Determine the maximum length
max_length = max(len(pre_Qs), len(post_Qs))

# Pad the shorter column with NaN values
if len(pre_Qs) < max_length:
    pre_Qs.extend([np.nan] * (max_length - len(pre_Qs)))

if len(post_Qs) < max_length:
    post_Qs.extend([np.nan] * (max_length - len(post_Qs)))

df_Qs = pd.DataFrame({'Pre': pre_Qs, 'Post': post_Qs})
df_Qs.to_csv('.\\add_data\Questions.csv', index = False)


In [10]:
## Merging table for DA - Technical knowledge. 
cols1 = list(df_Qs.loc[[0,1,7,8,9,10,11,12],'Pre'])
pre_tech = df_pre[cols1]
pre_tech.columns = ['ID', 'DA', 'NLP', 'Cyber', 
                    'VersionControl', 'Tableau', 'Python', 'Cloud']

cols2 = list(df_Qs.loc[[0,1,6,7,8,9,10,11],'Post'])
post_tech = df_post[cols2]
post_tech.columns = ['ID', 'DA', 'NLP', 'Cyber', 
                    'VersionControl', 'Tableau', 'Python', 'Cloud']

data_techDF = pd.merge(pre_tech, post_tech, 
                  on='ID', 
                  how='outer', suffixes = ('_pre', '_post'))
data_techDF.to_csv('.\\add_data\DA_Tech.csv')

In [11]:
# This code extracts the pre and post DA related tools that learner know

cols1 = list(df_Qs.loc[[0,4],'Pre'])
pre_tool = df_pre[cols1].copy()
pre_tool.columns = ['ID', 'DA_tools']
pre_1NF = normalize_1NF(pre_tool, 'DA_tools')
pre_1NF.to_csv('.\\add_data\DA_tools_Pre.csv', index = False)

cols2 = list(df_Qs.loc[[0,3],'Post'])
post_tool = df_post[cols2].copy()
post_tool.columns = ['ID', 'DA_learnTools']
post_1NF = normalize_1NF(post_tool, 'DA_learnTools')
post_1NF.to_csv('.\\add_data\DA_tools_Post.csv', index = False)

pre_PyTab = pre_1NF[(pre_1NF['DA_tools']=='Python') | (pre_1NF['DA_tools']=='Tableau')]
post_PyTab = post_1NF[(post_1NF['DA_learnTools']=='Python') | (post_1NF['DA_learnTools']=='Tableau')]

DA_toolDF = pd.merge(pre_PyTab, post_PyTab, 
                  on='ID', 
                  how='outer')
DA_toolDF.to_csv('.\\add_data\DA_tools.csv')


In [12]:
## career goals change 
cols1 = list(df_Qs.loc[[0,14],'Pre'])
pre_goal = df_pre[cols1].copy()
pre_goal.columns = ['ID', 'Career_goals']
pre_1NF = normalize_1NF(pre_goal, 'Career_goals')
pre_1NF.to_csv('.\\add_data\Career_goals_Pre.csv', index = False)


cols2 = list(df_Qs.loc[[0,13],'Post'])
post_goal = df_post[cols2].copy()
post_goal.columns = ['ID', 'Career_goals']
post_1NF = normalize_1NF(post_goal, 'Career_goals')
post_1NF.to_csv('.\\add_data\Career_goals_Post.csv', index = False)

DA_careerDF = pd.merge(pre_1NF, post_1NF, 
                  on='ID', 
                  how='outer', suffixes = ('_pre', '_post'))
DA_careerDF.to_csv('.\\add_data\DA_career.csv')

In [35]:
DA_tech_summary = data_techDF.iloc[:,1:].mean().to_frame().sort_index()
DA_tech_summary.reset_index(inplace = True)
DA_tech_summary.rename(columns={'index': 'DA_tools', 0: 'mean_value'}, inplace = True)
DA_tech_summary.to_csv('.\\add_data\DA_tech_sum.csv', index = False)